In [1]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import re

import warnings
warnings.filterwarnings("ignore")

MAX_KEY_SIZE = 26

In [2]:
np_dataset = np.genfromtxt(r'Datasets\base.txt', dtype=str)
np_password = np.ndarray(len(np_dataset), dtype="object")
np_user = np.ndarray(len(np_dataset), dtype="object")

In [3]:
def format_dataset(dataset: np.ndarray):
    pattern = "\|(.*?)\|"
    for i in tqdm(range(dataset.shape[0])):
        np_password[i] = re.search(pattern, np_dataset[i][1:]).group(1)
        np_user[i] = np_dataset[i].replace(np_password[i], "").replace(".", " ").replace("|", "")

In [4]:
def getMode():
    while 1:
        print('Do you want to encrypt or decrypt your database?')
        mode = input().lower()
        if mode in 'encrypt e decrypt d'.split():
            return mode
        else:
            print('You can type encrypt, e, decrypt or d only')

In [5]:
def getData():
    key = ''
    while 1:
        print('Do you want to encrypt/decrypt only the password or the entire base?')
        data = input().lower()
        if data in 'password p entire base base b'.split():
            return data
        else:
            print('You can type password, p, entire, base or b only')

In [6]:
def getKey():
    key = 0
    while True:
        print('Type a number between 1 and %s' % (MAX_KEY_SIZE))
        key = int(input())
        if (key >= 1 and key <= MAX_KEY_SIZE):
            return key


In [7]:
def getTranslatedMessage(mode, message, key):
    if mode[0] == 'd':
        key = -key
    translated = ''

    for symbol in message:
        if symbol.isalpha():
            num = ord(symbol)
            num += key

            if symbol.isupper():
                if num > ord('Z'):
                    num -= 26
                elif num < ord('A'):
                    num += 26
            elif symbol.islower():
                if num > ord('z'):
                    num -= 26
                elif num < ord('a'):
                    num += 26

            translated += chr(num)
        else:
            translated += symbol
    return translated

In [8]:
format_dataset(np_dataset)

100%|██████████████████████████████████████████████████████████████████████| 300000/300000 [00:00<00:00, 373672.24it/s]


In [9]:
mode = getMode()
data = getData()
key = getKey()

mode_crypt = ''    
for i in tqdm(range(np_dataset.shape[0])):
    if data in 'password p'.split():
        mode_crypt = 'Password'
        np_password[i] = getTranslatedMessage(mode, np_password[i], key)
    else:
        np_user[i] = getTranslatedMessage(mode, np_user[i], key)
        np_password[i] = getTranslatedMessage(mode, np_password[i], key)


Do you want to encrypt or decrypt your database?


 e


Do you want to encrypt/decrypt only the password or the entire base?


 d


You can type password, p, entire, base or b only
Do you want to encrypt/decrypt only the password or the entire base?


 b


Type a number between 1 and 26


 25


100%|██████████████████████████████████████████████████████████████████████| 300000/300000 [00:02<00:00, 116056.28it/s]


In [10]:
df = pd.DataFrame(np_user)

In [11]:
df.columns = ['User']
df['Password'] = np_password

In [12]:
df.to_csv(r'Datasets\pandas.txt', header=None, index=None, sep=' ', mode='w')